In [1]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import os
import dimod
from dwave.system import LeapHybridSampler

In [2]:
# --- Generate couplings ---
def generate_J(N, seed=None):
    # Create directory if it doesn't exist
    os.makedirs('generated_data', exist_ok=True)
    
    if seed is not None:
        np.random.seed(seed)
    std = 1.0 / np.sqrt(N)
    data = [(i, j, np.random.normal(0, std)) for i in range(N) for j in range(i+1, N)]
    df = pd.DataFrame(data, columns=['i','j','Jij'])
    
    if seed is not None:
        filepath = f'generated_data/Jij_N{N}_seed{seed}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")
    else:
        filepath = f'generated_data/Jij_N{N}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")

# --- Load couplings ---
def load_J(N, seed=None):
    filename = f'generated_data/Jij_N{N}_seed{seed}.csv' if seed is not None else f'generated_data/Jij_N{N}.csv'
    df = pd.read_csv(filename)
    N = max(df['i'].max(), df['j'].max()) + 1
    return [(int(i), int(j), float(Jij)) for i,j,Jij in df.values], N

In [4]:
def solve_SK(rows, N, seed, time_limit=None):
    from docplex.mp.model import Model
    import pandas as pd
    import os
    
    # Create directory if it doesn't exist
    os.makedirs('cplex_solutions', exist_ok=True)

    mdl = Model('SK_Ising')
    x = mdl.binary_var_list(N, name='x')

    expr = 0
    for i, j, Jij in rows:
        expr += -4*Jij * x[i]*x[j] + 2*Jij*(x[i] + x[j]) - Jij

    mdl.minimize(expr)

    if time_limit is not None:
        mdl.parameters.timelimit = float(time_limit)

    sol = mdl.solve(log_output=False)

    if not sol:
        print("No solution found.")
        return

    x_sol = [int(sol.get_value(var)) for var in x]
    S = [2*xi - 1 for xi in x_sol]
    E = sum(-Jij * S[i]*S[j] for i,j,Jij in rows)

    # Print solution status and gap
    status = mdl.solve_details.status
    mip_gap = mdl.solve_details.mip_relative_gap

    print(f"N = {N}\nEnergy = {E}")
    print(f"Status: {status}")
    if mip_gap is not None:
        print(f"MIP gap: {mip_gap:.4f}")
    # print(f"First spins: {S}")

    filepath = f'cplex_solutions/solution_N{N}_seed{seed}.csv'
    pd.DataFrame({'i': range(N), 'Energy_cplex': E, 'x': x_sol, 'S': S}).to_csv(filepath, index=False)
    print(f"Saved solution to {filepath}")
    return E, S

In [5]:
N = 50  # choose system size
energy_cplex_list = []
S_cplex_list = []
for i in range(500):
    generate_J(N, seed=i)
    rows, N = load_J(N, seed=i)
    energy_cplex, S_cplex = solve_SK(rows, N, seed=i, time_limit=300)
    energy_cplex_list.append(energy_cplex)
    S_cplex_list.append(S_cplex) 

Saved couplings for N=50 to generated_data/Jij_N50_seed0.csv
N = 50
Energy = -33.952800215526594
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed0.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed1.csv
N = 50
Energy = -36.2219649608972
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed1.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed2.csv
N = 50
Energy = -34.954109670887355
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed2.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed3.csv
N = 50
Energy = -35.93114968316619
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed3.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed4.csv
N = 50
Energy = -34.23939127574053
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/so

In [27]:
energy_cplex_array = np.array(energy_cplex_list)
energy_cplex_mean = np.mean(energy_cplex_array)
energy_cplex_std = np.std(energy_cplex_array)
print(f"Average CPLEX Energy over 500 instances: {energy_cplex_mean} ± {energy_cplex_std}") 

print("mean energy per site:", energy_cplex_mean / N)

Average CPLEX Energy over 500 instances: -35.315482216481094 ± 1.6055472305412284
mean energy per site: -0.7063096443296218


In [ ]:
ene_cplex_list = []
for l in range(500):
    df_result_cplex = pd.read_csv(f"cplex_solutions/solution_N50_seed{l}.csv")
    ene_cplex_list.append(df_result_cplex['Energy_cplex'][0])

In [29]:
ene_cplex_array = np.array(ene_cplex_list)
ene_cplex_mean = np.mean(ene_cplex_array)
ene_cplex_std = np.std(ene_cplex_array)
print(f"Average CPLEX Energy over 500 instances: {ene_cplex_mean} ± {ene_cplex_std}")

print("mean energy per site:", ene_cplex_mean / N)

Average CPLEX Energy over 500 instances: -35.315482216481094 ± 1.6055472305412286
mean energy per site: -0.7063096443296218


In [7]:
# # --- Build and solve D-Wave BQM model ---
# def solve_SK_dwave(rows, N, seed=None, time_limit=30):
#     # Create directory if it doesn't exist
#     os.makedirs('dwave_solutions', exist_ok=True)
#     # Build Binary Quadratic Model (BQM) in SPIN variables
#     h = {i: 0.0 for i in range(N)} # no onsite field
#     J = {(i, j): -Jij for i, j, Jij in rows} # match SK Hamiltonian: H = -sum Jij S_i S_j


#     bqm = dimod.BinaryQuadraticModel(h, J, dimod.SPIN)


#     sampler = LeapHybridSampler()
#     sampleset = sampler.sample(bqm, time_limit=time_limit)


#     result = sampleset.first.sample
#     S = [result[i] for i in range(N)]
#     energy = min(sampleset.record.energy)


#     print(f"N = {N}\nEnergy = {energy}") 
#     print(f"First spins: {S}")

#     filepath = f'dwave_solutions/solution_N{N}_seed{seed}.csv'

#     pd.DataFrame({'i': range(N), 'Energy_dwave': energy, 'S': S}).to_csv(filepath, index=False)

#     print(f"Saved solution to {filepath}")
#     return energy, S

In [8]:
# energy_dwave, S_dwave = solve_SK_dwave(rows, N, seed=seed, time_limit=30)